In [68]:
import pandas as pd
import numpy as np

np.random.seed(42)

# 1. Load HF dataset and convert to pandas
raw_df = pd.read_parquet("raw/FreshRetailNet-50K/train.parquet")
df_fresh_eval = pd.read_parquet("raw/FreshRetailNet-50K/eval.parquet")


In [69]:
# raw_df = raw_df.head(2000)
# 2. Rename for cleanliness
df = raw_df.rename(columns={
    "dt": "date",
    "sale_amount": "sales",
    "first_category_id": "category_1",
    "second_category_id": "category_2",
    "third_category_id": "category_3",
    "holiday_flag": "holiday",
    "precpt": "precip",
    "avg_temperature": "temp",
    "avg_humidity": "humidity",
    "avg_wind_level": "wind_level",
})

df["date"] = pd.to_datetime(df["date"])

# 3. Build a stable SKU-ID like you did for M5
#    here: city–store–product triple → id
df["id"] = "CID" + df["city_id"].astype(str) + "_SID" + df["store_id"].astype(str) + "_PID" + df["product_id"].astype(str) + "_MGID" + \
    df["management_group_id"].astype(str) + "_CAT1" + df["category_1"].astype(str) + "-CAT2" + df["category_2"].astype(str) + "-CAT3" + df["category_3"].astype(str)

In [70]:
def extract_daily_features(row):
    hs = np.array(row["hours_sale"], dtype=float)        # length 24
    st = np.array(row["hours_stock_status"], dtype=int)  # 1 = out-of-stock

    sale_hours = (hs > 0).sum()
    sale_hour_ratio = sale_hours / 24.0

    stockout_hours = st.sum()
    stockout_hour_ratio = stockout_hours / 24.0
    avail_hour_ratio = 1.0 - stockout_hour_ratio
    
    return pd.Series({
        "sale_hours": sale_hours,
        "sale_hour_ratio": sale_hour_ratio,
        "stockout_hours": stockout_hours,
        "stockout_hour_ratio": stockout_hour_ratio,
        "avail_hour_ratio": avail_hour_ratio,
    })

daily_feats = df.apply(extract_daily_features, axis=1)
df = pd.concat([df, daily_feats], axis=1)


In [71]:
tidy_df = df[['id',
    "date",
    "city_id", "store_id", "product_id",
    "management_group_id", "category_1", "category_2", "category_3",
    "sales",
    "sale_hours", "sale_hour_ratio",
    "stockout_hours", "stockout_hour_ratio", "avail_hour_ratio",
    "stock_hour6_22_cnt",
    "discount", "holiday", "activity_flag",
    "precip", "temp", "humidity", "wind_level",
]].copy()

In [72]:
g = tidy_df.groupby("id")["sales"]

summary = g.agg(["mean", "std", "count"])
summary = summary.rename(columns={"count": "T"})

summary["N"] = g.apply(lambda x: (x > 0).sum())
summary["ADI"] = summary["T"] / summary["N"].replace(0, 1)
summary["CV2"] = (summary["std"] / summary["mean"].replace(0, 1)) ** 2

summary["ADI_class"] = np.where(summary["ADI"] > 1.32, "High", "Low")
summary["CV2_class"] = np.where(summary["CV2"] > 0.49, "High", "Low")
summary["regime"] = summary["ADI_class"] + "-" + summary["CV2_class"]


In [73]:
tidy_df = tidy_df.merge(summary, on="id", how="left")

In [74]:
tidy_high_high = tidy_df[tidy_df["regime"] == "High-High"]
tidy_low_high  = tidy_df[tidy_df["regime"] == "Low-High"]
tidy_high_low   = tidy_df[tidy_df["regime"] == "High-Low"]
tidy_low_low   = tidy_df[tidy_df["regime"] == "Low-Low"]

def sample_by_regime(df_regime: pd.DataFrame, num_ids_needed: int) -> pd.DataFrame:
    """
    Sample num_ids_needed unique IDs from df_regime and return all their history.
    """
    concat_df = []
    for i, sku_id in enumerate(df_regime["id"].unique()):
        if i < num_ids_needed:
            concat_df.append(df_regime[df_regime["id"] == sku_id])
        else:
            break
    if not concat_df:
        return pd.DataFrame(columns=df_regime.columns)
    return pd.concat(concat_df, ignore_index=True)

df_high_high_sampled = sample_by_regime(tidy_high_high, 20)
df_low_high_sampled  = sample_by_regime(tidy_low_high, 10)
df_high_low_sampled   = sample_by_regime(tidy_high_low, 10)
df_low_low_sampled   = sample_by_regime(tidy_low_low, 10)

tidy_subset = pd.concat(
    [df_high_high_sampled, df_low_high_sampled, df_high_low_sampled, df_low_low_sampled],
    ignore_index=True
)

print(tidy_subset["regime"].value_counts())
print(tidy_subset["regime"].value_counts(normalize=True))


regime
High-High    1800
Low-High      900
Low-Low       900
Name: count, dtype: int64
regime
High-High    0.50
Low-High     0.25
Low-Low      0.25
Name: proportion, dtype: float64


C:\Users\topra\AppData\Local\Temp\ipykernel_31124\3370329160.py:25: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  tidy_subset = pd.concat(


In [83]:
print(tidy_df["regime"].value_counts())
print(tidy_df["regime"].value_counts(normalize=True))

regime
Low-Low      3649320
Low-High      709920
High-High     140760
Name: count, dtype: int64
regime
Low-Low      0.81096
Low-High     0.15776
High-High    0.03128
Name: proportion, dtype: float64


In [75]:
# Sort properly
tidy_subset = tidy_subset.sort_values(["id", "date"])

# Per-SKU day index (1..T within each id)
tidy_subset["day_idx"] = (
    tidy_subset
    .groupby("id")["date"]
    .rank(method="first")
    .astype(int)
)


In [76]:
# Sort properly
tidy_subset = tidy_subset.sort_values(["id", "date"])

# Per-SKU day index (1..T within each id)
tidy_subset["day_idx"] = (
    tidy_subset
    .groupby("id")["date"]
    .rank(method="first")
    .astype(int)
)


In [78]:
# a=b

In [79]:
# For 90-day series: use first 62 days as train, last 28 as test
TRAIN_HORIZON_END = 62   # 90 - 28
HORIZON = 14

train_df = tidy_subset[tidy_subset["day_idx"] <= TRAIN_HORIZON_END]
test_df  = tidy_subset[tidy_subset["day_idx"] > TRAIN_HORIZON_END]

# Inference input for LGBM (last 200 days equivalent; here min(200, series_len))
# For 90-day series you might just use last 62 or so; here we take last 62:
inference_input_df_lgbm = tidy_subset[
    tidy_subset["day_idx"] > (TRAIN_HORIZON_END - 62)
]

In [80]:
train_df.head()

,id,date,city_id,store_id,product_id,management_group_id,category_1,category_2,category_3,sales,...,mean,std,T,N,ADI,CV2,ADI_class,CV2_class,regime,day_idx
0,CID0_SID0_PID129_MGID6_CAT110-CAT233-CAT3181,2024-03-28,0,0,129,6,10,33,181,0.0,...,0.805556,0.894975,90,66,1.363636,1.234328,High,High,High-High,1
1,CID0_SID0_PID129_MGID6_CAT110-CAT233-CAT3181,2024-03-29,0,0,129,6,10,33,181,0.0,...,0.805556,0.894975,90,66,1.363636,1.234328,High,High,High-High,2
2,CID0_SID0_PID129_MGID6_CAT110-CAT233-CAT3181,2024-03-30,0,0,129,6,10,33,181,0.0,...,0.805556,0.894975,90,66,1.363636,1.234328,High,High,High-High,3
3,CID0_SID0_PID129_MGID6_CAT110-CAT233-CAT3181,2024-03-31,0,0,129,6,10,33,181,0.0,...,0.805556,0.894975,90,66,1.363636,1.234328,High,High,High-High,4
4,CID0_SID0_PID129_MGID6_CAT110-CAT233-CAT3181,2024-04-01,0,0,129,6,10,33,181,0.8,...,0.805556,0.894975,90,66,1.363636,1.234328,High,High,High-High,5


In [82]:
import os
os.makedirs("processed", exist_ok=True)

tidy_subset.to_csv("processed/freshretailnet_subset.csv", index=False)
train_df.to_csv("processed/train.csv", index=False)
test_df.to_csv("processed/test.csv", index=False)
inference_input_df_lgbm.to_csv(
    "processed/inference_input_df_lgbm.csv",
    index=False
)